# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Цель исследования: создать модель машинного обучения, которая сможет определить негативные комментарии на основе текста комментария.

План:
1. Загрузка данных
2. Обучение моделей
3. Выводы

Перед началом работы загрузим нужные библиотеки.

In [1]:
pip install imbalanced-learn -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install mlxtend -q

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandarallel -q

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install tqdm -q

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pandas jupyter pandarallel requests tqdm -q

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install nltk -q

Note: you may need to restart the kernel to use updated packages.


In [7]:
#pandas для работы с датафреймами
import pandas as pd
#математические библиотеки
import math
import numpy as np
#графики
from scipy import stats as st
import matplotlib.pyplot as plt
import seaborn as sns
#sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from mlxtend.feature_selection import ColumnSelector
from sklearn.naive_bayes import BernoulliNB
#работа с текстом
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import nltk
nltk.download('all')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import wordnet
#
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from pymystem3 import Mystem
import re
import torch
import transformers
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
#undersampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as Pipeline_imb
#
from pandarallel import pandarallel
from tqdm import tqdm
tqdm.pandas(desc="progress")
pandarallel.initialize(progress_bar = True)
from ipywidgets import interactive
from tqdm.notebook import tqdm_notebook

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\igsto\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\igsto\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\igsto\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\igsto\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\igsto\AppData\Roaming\nltk_data...
[

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


## 1. Загрузка и подготовка данных

Загрузим данные

In [8]:
try:
    df = pd.read_csv('/datasets/toxic_comments.csv') 
except:
    try:
        df = pd.read_csv('D://YandexPracticum//data//toxic_comments.csv') 
    except:
        df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv') 

In [9]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Удалим столбец Unnamed: 0 так как столбец просто дублирует индексы.

In [10]:
df = df.drop('Unnamed: 0', axis=1)
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


## 2. Векторизация текстов

### 2.1 Лемматизация

Зададим стоп-слова

In [11]:
stop_words = set(stopwords.words('english')) 

Далее необходимо избавиться от лишних символов вроде переноса строки, кавычек и т.д.

Заменим символы на пробелы.

In [18]:
df['token_text'] = df['text'].apply(lambda sentence: re.sub(r"[^a-z\']", ' ', sentence.lower()))

Зададим функцию для определение части речи

In [13]:
def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

Найдём часть речи для каждого слова.

In [19]:
df['token_text'] = df['token_text'].progress_apply(lambda sentence: nltk.pos_tag(nltk.word_tokenize(sentence)))
df['token_text']

progress: 100%|███████████████████████████████████████████████████████████████| 159292/159292 [12:26<00:00, 213.42it/s]


0         [(explanation, NN), (why, WRB), (the, DT), (ed...
1         [(d'aww, NN), (he, PRP), (matches, VBZ), (this...
2         [(hey, NN), (man, NN), (i, NN), ('m, VBP), (re...
3         [(more, RBR), (i, NNS), (ca, MD), (n't, RB), (...
4         [(you, PRP), (sir, VBP), (are, VBP), (my, PRP$...
                                ...                        
159287    [(and, CC), (for, IN), (the, DT), (second, JJ)...
159288    [(you, PRP), (should, MD), (be, VB), (ashamed,...
159289    [(spitzer, NN), (umm, JJ), (theres, VBZ), (no,...
159290    [(and, CC), (it, PRP), (looks, VBZ), (like, IN...
159291    [(and, CC), (i, VB), (really, RB), (do, VBP), ...
Name: token_text, Length: 159292, dtype: object

Преобразуем часть речи.

In [20]:
df['token_text'] = df['token_text'].apply(
    lambda sentence: list(map(lambda x: (x[0], pos_tagger(x[1])), sentence))
)

Лемматизируем.

In [23]:
lemmatizer = WordNetLemmatizer()

df['token_text'] = df['token_text'].progress_apply(
    lambda sentence: [(lambda word, tag: word if tag is None else lemmatizer.lemmatize(word, tag))(word, tag)
     for word, tag in sentence]
)    

progress: 100%|██████████████████████████████████████████████████████████████| 159292/159292 [00:46<00:00, 3454.43it/s]


Удалим стопслова

In [25]:
df['token_text'] = df['token_text'].apply(lambda x: [word for word in x if word not in stop_words])

In [26]:
df.head()

,text,toxic,token_text
0,Explanation\nWhy the edits made under my usern...,0,"[explanation, edits, make, username, hardcore,..."
1,D'aww! He matches this background colour I'm s...,0,"[d'aww, match, background, colour, 'm, seeming..."
2,"Hey man, I'm really not trying to edit war. It...",0,"[hey, man, 'm, really, try, edit, war, 's, guy..."
3,"""\nMore\nI can't make any real suggestions on ...",0,"[ca, n't, make, real, suggestion, improvement,..."
4,"You, sir, are my hero. Any chance you remember...",0,"[sir, hero, chance, remember, page, 's]"


### 2.2 Векторизация

Посмотрим на количество токсичных и нетоксичных комментариев.

In [27]:
df['toxic'].value_counts()

toxic
0    143106
1     16186
Name: count, dtype: int64

Нетоксичных комментариев примерно в 10 раз больше чем токсичных.

Далее необходимо посчитать tf-idf для каждого слова. Размер датафрейма довольно большой, а слов ещё больше. Необходимо как-то уменьшить количество слов. Можно найти частоту появления каждого слова и выбрать 2500 слов что встречаются чаще всего, после чего рассчитать tf-idf. Однако так как токсичных комментариев меньше необходимо найти слова, которые свойствены только токсичным комментариям и добавить эти слова к самым частоупотребляемым.

Посчитаем слова для каждой группы.

Сначала посчитаем слова для нетоксичных отзывов.

In [28]:
not_toxic_words = []

for sentence in df.loc[df['toxic'] == 0, 'token_text']:
    for word in sentence:
        not_toxic_words.append(word)

In [29]:
not_toxic_words = nltk.FreqDist(not_toxic_words)

Найдём количество слов

In [30]:
len(not_toxic_words)

153130

Создадим список со словами

In [31]:
not_toxic_words = list(not_toxic_words)

Проделаем аналогичную операцию для токсичной группы.

In [32]:
toxic_words = []

for sentence in df.loc[df['toxic'] == 1, 'token_text']:
    for word in sentence:
        toxic_words.append(word)

In [33]:
toxic_words = nltk.FreqDist(toxic_words)

In [34]:
len(toxic_words)

29431

Краткий осмотр показывает, что некоторые слова встречаются как в токсичной группе, так и в нетоксичной, при этом нельзя исключать обратной ситуации. Здравый смысл подсказывает, что токсичных слов в нетоксичной группе будет мало, поэтому возьмём первые 2500 слов из нетоксичной группы, а потом уберём эти слова из токсичной.

In [35]:
not_toxic_words = not_toxic_words[:2500]

In [36]:
toxic_words = list(toxic_words)
toxic_words

['fuck',
 "n't",
 'suck',
 "'s",
 'go',
 'like',
 'wikipedia',
 'shit',
 'nigger',
 'get',
 'u',
 'page',
 'know',
 'hate',
 'faggot',
 "'re",
 'bitch',
 'gay',
 'make',
 'die',
 'article',
 'people',
 'block',
 'say',
 'fat',
 'moron',
 'talk',
 'cunt',
 'user',
 'one',
 'think',
 'edit',
 'hi',
 "'m",
 'jew',
 'stop',
 'want',
 'stupid',
 'dick',
 'wiki',
 'time',
 'pig',
 'cock',
 'see',
 'would',
 'take',
 'penis',
 "'",
 'life',
 'delete',
 'fucking',
 'right',
 'bullshit',
 'idiot',
 'good',
 'asshole',
 'even',
 'give',
 'come',
 'use',
 'fag',
 'dont',
 'wanker',
 'bad',
 'please',
 'ban',
 'ball',
 'try',
 'tell',
 'bark',
 'vandalism',
 'well',
 'thing',
 'sex',
 'kill',
 'love',
 'need',
 'little',
 'call',
 'really',
 'piece',
 'look',
 'care',
 'eat',
 'ass',
 'nipple',
 'also',
 'hey',
 'hell',
 'ca',
 'bastard',
 'way',
 'aid',
 'back',
 'write',
 'fucker',
 'dickhead',
 'guy',
 'keep',
 'big',
 'post',
 'admin',
 'remove',
 'nothing',
 'leave',
 "'ll",
 "'ve",
 'damn',


In [37]:
toxic_words = [word for word in toxic_words if word not in not_toxic_words]
toxic_words

['fuck',
 'suck',
 'shit',
 'nigger',
 'faggot',
 'bitch',
 'fat',
 'moron',
 'cunt',
 'dick',
 'pig',
 'cock',
 'penis',
 'fucking',
 'bullshit',
 'asshole',
 'fag',
 'wanker',
 'bark',
 'ass',
 'nipple',
 'bastard',
 'fucker',
 'dickhead',
 'shut',
 'loser',
 'fucksex',
 'yourselfgo',
 'homo',
 'cocksucker',
 'poop',
 'buttsecks',
 'mothjer',
 'pussy',
 'fggt',
 'noobs',
 'bastered',
 'lick',
 'dumb',
 'assad',
 'anal',
 'heil',
 'mexican',
 'pathetic',
 'boob',
 'fuckin',
 'hanibal',
 'offfuck',
 'ur',
 'fart',
 'niggas',
 'sexsex',
 "'fuck",
 'whore',
 'notrhbysouthbanof',
 'retard',
 'cocksucking',
 'oxymoron',
 'piss',
 'prick',
 'criminalwar',
 'bunksteve',
 'chester',
 'bum',
 'mitt',
 'romney',
 'marcolfuck',
 'retarded',
 'bloody',
 'dirty',
 'jerk',
 'arse',
 'homeland',
 'tommy',
 'cheese',
 'fack',
 'vomit',
 'securityfuck',
 'cuntbag',
 'cougar',
 'youbollocks',
 'faggots',
 'veggietales',
 'atheist',
 'egg',
 'edgar',
 'ancestryfuck',
 'ugly',
 'vagina',
 'nigga',
 'dela

Найдём длину списка токсичных слов

In [38]:
len(toxic_words)

26946

Возьмём первые 500

In [39]:
toxic_words = toxic_words[:500]

In [40]:
not_toxic_words = not_toxic_words[1:]
not_toxic_words

["'s",
 'page',
 "n't",
 'wikipedia',
 'talk',
 'use',
 'would',
 'one',
 'please',
 'edit',
 'make',
 'like',
 'see',
 'say',
 'think',
 'source',
 'know',
 'also',
 'get',
 'add',
 'time',
 'go',
 'people',
 'user',
 "'m",
 'good',
 'may',
 'need',
 'link',
 'name',
 'image',
 'take',
 'block',
 "'",
 'find',
 'delete',
 'remove',
 'want',
 'well',
 'look',
 'work',
 'thanks',
 'even',
 'could',
 'help',
 'list',
 'comment',
 'deletion',
 'change',
 'information',
 'section',
 'question',
 'way',
 "'ve",
 'point',
 'write',
 'editor',
 'give',
 'wp',
 'first',
 'try',
 'new',
 'thank',
 'thing',
 'fact',
 'seem',
 'state',
 'discussion',
 'reference',
 'read',
 'place',
 'ask',
 'many',
 'right',
 'much',
 'revert',
 'edits',
 'include',
 'create',
 'tag',
 'mean',
 'really',
 'since',
 'note',
 'come',
 'reason',
 'policy',
 'issue',
 'content',
 "'re",
 'two',
 'show',
 'someone',
 'back',
 'call',
 'word',
 'year',
 'post',
 'case',
 'still',
 'consider',
 'leave',
 'mention',
 "'

Объединим токсичные и нетоксичные слова.

In [41]:
valid_words = not_toxic_words + toxic_words
len(valid_words)

2999

Преобразуем колонку token_text.

In [42]:
df['token_text_valid'] = df['token_text'].apply(lambda sentence: [word for word in sentence if word in valid_words])

In [43]:
df['token_text_valid'] = df['token_text_valid'].apply(lambda sentence: ' '.join([word for word in sentence]))

Найдём количество отзывов, в которых нет выбранных слов для каждой группы.

In [44]:
df.loc[((df['token_text_valid'] == '') | (df['token_text_valid'] == ' ')) & (df['toxic'] == 0), 'toxic'].count()

562

In [45]:
df.loc[((df['token_text_valid'] == '') | (df['token_text_valid'] == ' ')) & (df['toxic'] == 1), 'toxic'].count()

54

Количество таких отзывов мало относительно всей таблицы, поэтому их можно убрать.

In [46]:
df = df.loc[~((df['token_text_valid'] == '') | (df['token_text_valid'] == ' '))]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 158676 entries, 0 to 159291
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   text              158676 non-null  object
 1   toxic             158676 non-null  int64 
 2   token_text        158676 non-null  object
 3   token_text_valid  158676 non-null  object
dtypes: int64(1), object(3)
memory usage: 6.1+ MB


Сделаем dataframe из преобразованного текста для корректной работы метода fit моделей.

In [47]:
df_text = pd.DataFrame(df['token_text_valid'])

In [48]:
df_text.head()

,token_text_valid
0,explanation edits make username fan revert n't...
1,match background colour 'm stick thanks talk j...
2,hey man 'm really try edit war 's guy constant...
3,ca n't make real suggestion improvement wonder...
4,sir hero chance remember page 's


## 3. Обучение моделей

### 3.1 Разделение данных.

Разделим данные на тренировочную и тестовую выборки.

In [49]:
X_train, X_test, y_train, y_test = train_test_split(
    df_text,
    df['toxic'],
    random_state = 42
)

### 3.2 Составление пайплайнов и обучение моделей

Далее необходимо составить пайплайны для ML модели. Чтобы нелинейные модели работали быстро учтём дисбаланс классов и применим undersampling.

Составим пайплайны для моделей.

In [50]:
pipeline_imb = Pipeline_imb([
    #('sample', RandomUnderSampler()),
    ('col_selector', ColumnSelector(cols=('token_text_valid'),drop_axis=True)),
    ('vect', TfidfVectorizer()),
    ('model', LogisticRegression(random_state = 42)
    )
])

In [51]:
param_grid = [
    {
        'model': [LinearSVC()]
        #'model__kernel': ['linear']
    },
    {
        'model': [LogisticRegression(random_state = 42)],
        'model__C': range(1, 10)
    },
    {
        'model': [DecisionTreeClassifier(random_state=42)],
        'model__max_depth': range(2, 5),
        'model__min_samples_split': range(2, 5),
        'model__min_samples_leaf': range(1, 5),
    },
    {
        'model': [BernoulliNB()]
    }
]

In [52]:
gs = GridSearchCV(
    pipeline_imb,
    param_grid,
    n_jobs=-1,
    scoring='f1',
    cv=5,
    error_score='raise',
    verbose=10 #специально поставил чтобы видеть время обучения
)

In [53]:
gs.fit(X_train, y_train)

Fitting 5 folds for each of 47 candidates, totalling 235 fits


GridSearchCV(cv=5, error_score='raise',
             estimator=Pipeline(steps=[('col_selector',
                                        ColumnSelector(cols='token_text_valid',
                                                       drop_axis=True)),
                                       ('vect', TfidfVectorizer()),
                                       ('model',
                                        LogisticRegression(random_state=42))]),
             n_jobs=-1,
             param_grid=[{'model': [LinearSVC()]},
                         {'model': [LogisticRegression(random_state=42)],
                          'model__C': range(1, 10)},
                         {'model': [DecisionTreeClassifier(random_state=42)],
                          'model__max_depth': range(2, 5),
                          'model__min_samples_leaf': range(1, 5),
                          'model__min_samples_split': range(2, 5)},
                         {'model': [BernoulliNB()]}],
             scoring='f1', verbose=10)

Найдём лучшую метрику на кросс-валидации.

In [54]:
gs.best_score_

0.7799227720719295

Найдём метрику на тестовой выборке.

In [55]:
model = gs.best_estimator_

In [56]:
pred = model.predict(X_test)

In [57]:
f1_score(y_test, pred)

0.777333517165311

## 4. Выводы

В задаче было необходимо составить модель, которая будет отличать токсичные комментарии от нетоксичных. Для достижения этой цели были отобраны токсичные и нетоксичные слова с помощью топа самых встречающихся слов, после чего каждый комментарий был очищен от остальных слов. В качестве модели были использованы логистическая регрессия и Linear SVC. Лучшей моделью оказалась модель Linear SVC, f1 мера на кросс-валидации оказалась равна 0.777, на тестовой выборке 0.775.